In [1]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
data_path = 'data/'
df_train = pd.read_csv('http://bit.ly/kaggletrain')
df_test = pd.read_csv('http://bit.ly/kaggletrain')
df_train.shape

(891, 12)

In [2]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,NaN,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,NaN,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,NaN,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,NaN,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,NaN,373450


In [3]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,3
2,object,5


In [4]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Parch', 'Pclass', 'SibSp']

3 Float Features : ['Age', 'Fare', 'Survived']

5 Object Features : ['Cabin', 'Embarked', 'Name', 'Sex', 'Ticket']


### 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?

In [6]:
df[int_features].mean()   

Parch     0.381594
Pclass    2.308642
SibSp     0.523008
dtype: float64

In [7]:
df[int_features].max()

Parch     6
Pclass    3
SibSp     8
dtype: int64

In [8]:
df[int_features].nunique()

Parch     7
Pclass    3
SibSp     7
dtype: int64

In [9]:
df[float_features].mean()

Age         29.699118
Fare        32.204208
Survived     0.383838
dtype: float64

In [11]:
df[float_features].max()

Age          80.0000
Fare        512.3292
Survived      1.0000
dtype: float64

In [10]:
df[float_features].nunique()

Age          88
Fare        248
Survived      2
dtype: int64

In [12]:
df[object_features].mean()

Series([], dtype: float64)

In [13]:
df[object_features].max()

Series([], dtype: float64)

In [14]:
df[object_features].nunique()

Cabin       147
Embarked      3
Name        891
Sex           2
Ticket      681
dtype: int64

### 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?

### 所以三大類特徵中，哪一大類處理起來應該最複雜?

In [5]:
# 例 : 整數 (int) 特徵取平均 (mean)
df[int_features].mean()

Parch     0.381594
Pclass    2.308642
SibSp     0.523008
dtype: float64

object型態不能比大小或是找出平均值
不過object 仍有部分欄位可傳回最大值，可以知道 object 欄位超過一定長度時會當成字串，此時會以字典順序做大小比較

複數 (a+bi)，因為需要兩個數字表示，雖然概念上是數值，但因為無法比較大小，因此只能歸類為"類別型欄位"
不論其他更奇怪的例子，都可以歸類為類別型欄位，因此類別型欄位應該最難處理